<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Addestramento_modello_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [1]:
#Librerie

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import shutil
import copy


install torchbearer

In [2]:
try:
    import torchbearer
except:
    !pip install -q torchbearer
    import torchbearer
    
print(torchbearer.__version__)

     |████████████████████████████████| 138 kB 2.3 MB/s 
0.5.3


##Check device


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

1.9.0+cu102
False


RuntimeError: ignored

##Def variabili

In [ ]:
#salvataggio modello
model_save_name = 'model_DenseNet201'
path_model_save = F"/content/gdrive/My Drive/ModelliCNN/{model_save_name}" 

##Collegamento google drive

In [ ]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

#**Dataset**

##Caricamento dataset e creazione Dataloader

In [3]:
import torch
import torchvision
from torchvision import transforms

from torchbearer.cv_utils import DatasetValidationSplitter

BATCH_SIZE = 128

data_transform=transforms.Compose([transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

trainset = torchvision.datasets.STL10('/stl10',split='train',transform=data_transform,download=True)
testset = torchvision.datasets.STL10('/stl10',split='test',transform=data_transform,download=True)

#splitter = DatasetValidationSplitter(len(trainset), 0.2)
#trainset_split = splitter.get_train_dataset(trainset)
#valset = splitter.get_val_dataset(trainset)

#traingen = torch.utils.data.DataLoader(trainset_split, pin_memory=True, batch_size=BATCH_SIZE, shuffle=True,num_workers=2)
#valgen = torch.utils.data.DataLoader(valset, pin_memory=True, batch_size=BATCH_SIZE, shuffle=True,num_workers=2)

#testgen = torch.utils.data.DataLoader(testset, pin_memory=True, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting /stl10/stl10_binary.tar.gz to /stl10
Files already downloaded and verified


In [4]:
#riduzione train set
trainset.data=trainset.data[:100]

In [5]:
dataloaders={'train':traingen,'valid':valgen,'test':testgen}
image_datasets={'train':trainset_split,'valid':valset,'test':testset}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}

NameError: ignored

##Stampa dimensioni

In [ ]:
print("Dimensione trainset_split: "+str(len(trainset_split)))
print("Dimensione valset: "+str(len(valset)))
print("Dimensione testset: "+str(len(testset)))

##Nomi classi

In [6]:
classes_name=trainset.classes
class_number=len(classes_name)
print(classes_name)
print(class_number)

['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
10


#SpinalNet

In [ ]:
#model_ft = models.vgg19_bn(pretrained=True)
#num_ftrs = model_ft.classifier[0].in_features

model_ft = models.wide_resnet101_2(pretrained=True)
num_ftrs = model_ft.fc.in_features

half_in_size = round(num_ftrs/2)
layer_width = 1024
Num_class=10

class SpinalNet(nn.Module):
    def __init__(self):
        super(SpinalNet, self).__init__()
        
        self.fc_spinal_layer1 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_in_size, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_spinal_layer2 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_in_size+layer_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_spinal_layer3 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_in_size+layer_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_spinal_layer4 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(half_in_size+layer_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_out = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(layer_width*4, Num_class),)
        
        

    def forward(self, x):
        x1 = self.fc_spinal_layer1(x[:, 0:half_in_size])
        x2 = self.fc_spinal_layer2(torch.cat([ x[:,half_in_size:2*half_in_size], x1], dim=1))
        x3 = self.fc_spinal_layer3(torch.cat([ x[:,0:half_in_size], x2], dim=1))
        x4 = self.fc_spinal_layer4(torch.cat([ x[:,half_in_size:2*half_in_size], x3], dim=1))
        
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.cat([x, x3], dim=1)
        x = torch.cat([x, x4], dim=1)

        
        x = self.fc_out(x)
        return x

class SpinalNet_Resnet(nn.Module):
    def __init__(self):
        super(SpinalNet_Resnet, self).__init__()
        
        self.layer1 = nn.Linear(half_in_size, layer_width)
        self.layer2 = nn.Linear(half_in_size+layer_width, layer_width)
        self.layer3 = nn.Linear(half_in_size+layer_width, layer_width)
        self.layer4 = nn.Linear(half_in_size+layer_width, layer_width)
        self._out = nn.Linear(layer_width*4, Num_class)
        
    def forward(self, x):
        x1 = self.layer1(x[:, 0:half_in_size])
        x2 = self.layer2(torch.cat([ x[:,half_in_size:2*half_in_size], x1], dim=1))
        x3 = self.layer3(torch.cat([ x[:,0:half_in_size], x2], dim=1))
        x4 = self.layer4(torch.cat([ x[:,half_in_size:2*half_in_size], x3], dim=1))
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.cat([x, x3], dim=1)
        x = torch.cat([x, x4], dim=1)

        x = self._out(x)
        return x

net_fc = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, Num_class)
        )


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    test_token=0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid', 'test']:
             
            
            '''
            Test when a better validation result is found
            '''
            if test_token ==0 and phase == 'test':
                continue
            test_token =0
            
            
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                test_token =1


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
'''
Changing the fully connected layer to SpinalNet
'''

model_ft.fc = SpinalNet_Resnet()#nn.Linear(num_ftrs, 10)
#model_ft.classifier = SpinalNet()



model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

In [ ]:
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

# **Creazione modello**

##Load model pre-trained

In [7]:
model_conv = models.resnet50(pretrained=True)
for param in model_conv.parameters():
  param.requires_grad = False #non modifico i parametri della parte riguardante le features durante il training, poiché voglio usare quella parte così com'è stata pre-addestrata

print(model_conv)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

##Aggiunta classificatore e freeze parametri

In [8]:
model_conv.fc=nn.Linear(2048,class_number)

for param in model_conv.fc.parameters():
  param.requires_grad = True #invece pongo il calcolo del gradiente del classificatore, ovvero attivo il calcolo dei pesi

#**Training**

##Def train definitiva

In [82]:
from collections import OrderedDict
from functools import partial
import sys

import numpy as np
import sys
import torch

from functools import partial
from torch.utils.data import DataLoader, TensorDataset


##### Data utils #####

def log_to_message(log, precision=4):
    fmt = "{0}: {1:." + str(precision) + "f}"
    return "    ".join(fmt.format(k, v) for k, v in log.items())


class ProgressBar(object):
    """Cheers @ajratner"""

    def __init__(self, n, length=40):
        # Protect against division by zero
        self.n      = max(1, n)
        self.nf     = float(n)
        self.length = length
        # Precalculate the i values that should trigger a write operation
        self.ticks = set([round(i/100.0 * n) for i in range(101)])
        self.ticks.add(n-1)
        self.bar(0)

    def bar(self, i, message=""):
        """Assumes i ranges through [0, n-1]"""
        if i in self.ticks:
            b = int(np.ceil(((i+1) / self.nf) * self.length))
            sys.stdout.write("\r[{0}{1}] {2}%\t{3}".format(
                "="*b, " "*(self.length-b), int(100*((i+1) / self.nf)), message
            ))
            sys.stdout.flush()

    def close(self, message=""):
        # Move the bar to 100% before closing
        self.bar(self.n-1)
        sys.stdout.write("{0}\n\n".format(message))
        sys.stdout.flush()



def training_f(trainset, numEpochs, model_conv, criterionCNN, optimizer_conv,
                          best_acc, best_loss, best_epoca,batch_size, validation_split=0):
  
  best_model_wts = copy.deepcopy(model_conv.state_dict())
  
  traingen = torch.utils.data.DataLoader(trainset, pin_memory=True, batch_size=batch_size, shuffle=True,num_workers=2)
    
  if validation_split:
      splitter = DatasetValidationSplitter(len(trainset), validation_split)
      trainset = splitter.get_train_dataset(trainset)
      valset = splitter.get_val_dataset(trainset)
      print(len(valset))
      traingen = torch.utils.data.DataLoader(trainset, pin_memory=True, batch_size=batch_size, shuffle=True,num_workers=2)
      valgen = torch.utils.data.DataLoader(valset, pin_memory=True, batch_size=batch_size)
      print("len traingen: "+str(len(traingen)))
      print("len valgen: "+str(len(valgen)))
  for epochs in range(1,numEpochs + 1):
    since = time.time()
    
    print("Epoch {0} / {1}".format(epochs, numEpochs))
    pb = ProgressBar(len(trainset))
    log = OrderedDict()
    modelLoss_train = 0.0
    modelAcc_train = 0.0

    model_conv.train() 

    totalSize = 0
    batch_i=0
    #for each batch: operazioni per gli algoritmi di ottimizzazione
    for inputs,labels in traingen:
      #inputs = inputs.type(torch.FloatTensor).cuda()
      #labels = labels.cuda()
      
      batch_i=batch_i+1

      optimizer_conv.zero_grad()
      model_conv.zero_grad()
      
      y = model_conv(inputs)
      outp, preds = torch.max(y, 1)   
        
      lossCNN = criterionCNN(y, labels) #media per batch

      modelLoss_train += lossCNN.item() * inputs.size(0)
      totalSize += inputs.size(0)
      modelAcc_train += torch.sum(preds == labels.data).item()

      lossCNN.backward()  # pred = f(x)   -> loss = L(f(x), l_true)
      
      optimizer_conv.step()
      
      log['loss'] =modelLoss_train
      log['acc']=modelAcc_train
      pb.bar(batch_i, log_to_message(log))
    
      #print("modelACC_train: "+str(modelAcc_train))
    #calcolo loss e accuracy
    modelLoss_epoch_train = modelLoss_train/totalSize
    modelAcc_epoch_train  = modelAcc_train/totalSize
    
    #print('TRAIN on %d [Loss: %.4f  ACC: %.4f]' 
     #     %(totalSize, modelLoss_epoch_train, modelAcc_epoch_train))

    if validation_split:    
        #validation
        model_conv.eval()
        totalSize_val = 0
        modelLoss_val = 0.0
        modelAcc_val = 0.0
        del inputs,labels  

        for inputs,labels in valgen:
          print("validation")
          #input = input.type(torch.FloatTensor).cuda()
          #label = label.cuda()
          
          y = model_conv(inputs)
          outp, preds = torch.max(y, 1)
          lossCNN = criterionCNN(y, labels)

          modelLoss_val += lossCNN.item() * inputs.size(0)
          totalSize_val += inputs.size(0)
          modelAcc_val += torch.sum(preds == labels.data).item()        

        modelLoss_epoch_val=modelLoss_val/totalSize_val
        modelAcc_epoch_val = modelAcc_val/totalSize_val
        time_elapsed = time.time()-since

        log['loss_val'] =modelLoss_epoch_val
        log['acc_val']=modelAcc_epoch_val
        #print("modelACC_val: "+str(modelAcc_epoch_val))
    #print('\rVAL on %d [Loss: %.4f  ACC: %.4f] [TIME: %.0f m %.0f s]' 
     #     %(totalSize_val, modelLoss_epoch_val, 
      #      modelAcc_epoch_val, time_elapsed // 60, time_elapsed % 60))
    
    if (modelAcc_epoch_val > best_acc) or (modelAcc_epoch_val == best_acc and modelLoss_epoch_val < best_loss) :
      log['.... Saving best weights ....']
      best_acc = modelAcc_epoch_val
      best_loss = modelLoss_epoch_val
      best_epoca = epochs
      best_model_wts = copy.deepcopy(model_conv.state_dict())

    pb.close(log_to_message(log))

  model_conv.load_state_dict(best_model_wts)
  return model_conv




##Set iperparametri

In [83]:
#iper-parametri iniziali
learning_rate = 0.0001
num_epoch = 10

startEpoch =1
best_acc = 0
best_loss=0
best_epoca = 0
#richiamo funzione che effettua il training con validation

#model_conv = model_conv.cuda() #sposta i calcoli sulla gpu
criterion = nn.CrossEntropyLoss() #criterio dell'aggiornamento del gradiente: minimizzazione funzione loss entropia
optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=learning_rate) #ottimizzatore: adam

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

##Addestramento

In [84]:
best_model=training_f(trainset, num_epoch, model_conv, criterion, optimizer_conv,best_acc, best_loss, best_epoca,batch_size=1, validation_split=0.2)

20
len traingen: 80
len valgen: 20
Epoch 1 / 10
[=========================================] 101%	loss: 191.9158    acc: 13.0000

IndexError: ignored

#**Testing**

In [ ]:
value_preds=model.predict(x_test) #contiene i valori tra 0 e 1 predetti per ognuna delle 10 classi e per ogni immagine
preds = np.argmax(value_preds, axis=1) #(le predizioni vanno da 0 a 9 e indicano la classe predetta)
acc = (np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0])*100
print('\033[1m'+"Accuracy on test set:", (round(acc,2)))

Accuracy on test set: 76.24


#**Salvataggio modello**

In [ ]:
#salva modello su drive
model.save(path_model_save)